In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Formula para el target
t = tiempo ejecución promedio\
e = error relativo promedio\
w1 = peso de t\
w2 = peso de e

Si t > 0 ((log_10(t)-3) / 3) x W1 + e x W2 \
Si t == 0 ((t-3)/3) x W1 + e x W2

### Cargar archivos

In [1]:
import pandas as pd
import os
import glob
import math as mt

In [17]:
#def generate_target(t, e, w1, w2):
#    fix = 0
#    if t != 0: 
#        fix = mt.log10(t/10) #Considerando que todos los tiempos promedios dan en milisegundos
#    else: fix = t
#    if fix-3.0 >= 0:
#        return (((fix-3.0)/3.0)*w1 + e*w2)
#    else:
#        return ((-1*(fix-3.0)/3.0)*w1 + e*w2)
def generate_target2(t0,t,e,k,w1,w2):
    t0 = t0/1000
    t = t/1000
    if k*(t-t0) > 709:
        return 0
    elif k*(t-t0) < -709:
        return 1
    else:
        return (1/(1+mt.exp(k*(t-t0))))*w1 + (1-e)*w2
        
def generate_binary_label(score, threshold=0.5):
    if score >= threshold:
        return 1
    else:
        return 0

In [3]:
#df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
#df = pd.concat(map(pd.read_csv,["./data/kp/datos1.csv","./data/kp/datos2.csv"]))
df2 = pd.read_csv("./data/kp/datos-50-1000.csv", dtype={"Generacion": int, "Numero de nodos": int, "Numero de funciones":int, "Numero de terminales":int, 
                                                 "Iteraciones Promedio DoWhile":int, "Profundidad":int, "Terminal Agregar":int, "Terminal Eliminar":int, "Terminal Mejora":int,
                                                 "ErrorRelativoPromedio":float,"Fitness":float,"Tiempo de Ejecucion Promedio(ms)":float })

In [4]:
print(df2)

        Generacion  Numero de Nodos  Numero de Terminales  \
0                0                4                     3   
1                0                2                     1   
2                0                7                     4   
3                0                4                     2   
4                0               10                     4   
...            ...              ...                   ...   
999995         999              352                   171   
999996         999              364                   177   
999997         999              326                   160   
999998         999              280                   136   
999999         999              339                   167   

        Numero de Funciones  Iteraciones Promedio DoWhile  Terminal Agregar  \
0                         1                             0                 1   
1                         1                             0                 0   
2                         3   

In [18]:
#df2 = df2.drop('Node Name', axis=1)
#df2
generate_target2(50.0, 45000.0,0.03,10,0.6, 0.4)
#df4 = df2.sort_values('Tiempo(ms)')
#df4.to_csv('./data/kp/datos-1000-sorted.csv', index = False)
#df4

0.388

In [12]:
df2['score_aceptabilidad'] = df2.apply(lambda row: generate_binary_label(generate_target2(50.0,row["Tiempo(ms)"], 
                                                                     row['ErrorRelativoPromedio'],10,
                                                                     0.6,0.4)), axis=1)

In [13]:
df2['score_aceptabilidad']

0         0
1         0
2         0
3         0
4         0
         ..
999995    1
999996    1
999997    1
999998    1
999999    1
Name: score_aceptabilidad, Length: 1000000, dtype: int64

In [14]:
df3 = df2.drop(['Tiempo(ms)', 'ErrorRelativoPromedio'], axis=1)
display(df3)

,Generacion,Numero de Nodos,Numero de Terminales,Numero de Funciones,Iteraciones Promedio DoWhile,Terminal Agregar,Terminal Eliminar,Terminal Mejora,Profundidad,score_aceptabilidad
0,0,4,3,1,0,1,0,1,2,0
1,0,2,1,1,0,0,0,1,2,0
2,0,7,4,3,200,2,1,1,3,0
3,0,4,2,2,0,0,2,0,3,0
4,0,10,4,6,0,1,1,0,5,0
...,...,...,...,...,...,...,...,...,...,...
999995,999,352,171,181,1400,83,49,7,20,1
999996,999,364,177,187,1400,86,54,7,20,1
999997,999,326,160,166,1800,70,50,7,20,1
999998,999,280,136,144,1000,68,34,8,20,1


In [19]:
df3.to_csv('./data/cgp/set-1000.csv', index=False)